In [1]:
using DataFramesMeta
using StatsBase
using CSV

dir = "/Users/fujisetakumi/_private/data_science/100knocks-preprocess/data/"
df_customers  = CSV.read(dir * "customers.csv", DataFrame)
df_categories = CSV.read(dir * "categories.csv", DataFrame)
df_products   = CSV.read(dir * "products.csv", DataFrame)
df_geocodes   = CSV.read(dir * "geocodes.csv", DataFrame)
df_receipts   = CSV.read(dir * "receipts.csv", DataFrame)
df_stores     = CSV.read(dir * "stores.csv", DataFrame)
;
ENV["COLUMNS"] = 1000
;

In [2]:
# P-076 層化抽出(10%)
stats = @chain df_customers begin
    groupby(:gender_cd)
    combine(:gender_cd => length => :count)
    @rtransform(:sample_limit = :count ÷ 10)
    select(Not(:count))
end

customers_10pct = @chain df_customers begin
    groupby(:gender_cd)
    @transform(:row_in_gender = 1:length(:gender_cd))
    innerjoin(stats, on= :gender_cd => :gender_cd, makeunique=true)
    @rsubset(:row_in_gender ≤ :sample_limit)
    sort([:gender_cd, :row_in_gender])
    select(Not([:row_in_gender, :sample_limit]))
end

@chain customers_10pct begin
    groupby(:gender_cd)
    combine(:gender_cd => length)
end

Row,gender_cd,gender_cd_length
,Int64,Int64
1,0,298
2,1,1791
3,9,107


In [3]:
# P-077 外れ値の抽出(対数化した上で平均と標準偏差を計算し、3σを超えたものを抽出)
@chain df_receipts begin
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_sum_log = log.(:amount_sum))
    @transform(:std = zscore(:amount_sum_log))
    @rsubset(abs(:std) > 3)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_log,std
,String15,Int64,Float64,Float64
1,ZZ000000000000,12395003,16.3328,7.96361


In [4]:
# P-078 外れ値の抽出(第1四分位と第3四分位の差であるIQRを用いて抽出)
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @aside quartiles = quantile(_.amount_sum)
    @aside IQR = quartiles[4] - quartiles[2]
    @rsubset(:amount_sum < (quartiles[2] - 1.5*IQR)
        || (quartiles[4] + 1.5*IQR) < :amount_sum)
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum
,String15,Int64
1,CS001414000048,8584
2,CS001605000009,18925
3,CS002415000594,9568
4,CS004414000181,9584
5,CS005415000137,8734
6,CS006414000001,9156
7,CS006414000029,9179
8,CS006415000105,10042
9,CS006415000147,12723


In [5]:
# P-079 欠損値の確認
# filter(x -> any(ismissing, x), df_products)
describe(df_products, :nmissing)

Row,variable,nmissing
,Symbol,Int64
1,product_cd,0
2,category_major_cd,0
3,category_medium_cd,0
4,category_small_cd,0
5,unit_price,7
6,unit_cost,7


In [6]:
# P-080 欠損値を除外
@show nrow(df_products)
dropmissing(df_products)

nrow(df_products) = 10030


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
,String15,Int64,Int64,Int64,Int64,Int64
1,P040101001,4,401,40101,198,149
2,P040101002,4,401,40101,218,164
3,P040101003,4,401,40101,230,173
4,P040101004,4,401,40101,248,186
5,P040101005,4,401,40101,268,201
6,P040101006,4,401,40101,298,224
7,P040101007,4,401,40101,338,254
8,P040101008,4,401,40101,420,315
9,P040101009,4,401,40101,498,374


In [7]:
# P-081 欠損値を平均値で補完
@chain df_products begin
    @aside mean_price = round(Int, mean(skipmissing(_.unit_price)))
    @aside mean_cost = round(Int, mean(skipmissing(_.unit_cost)))
    @rtransform(
        :unit_price = coalesce(:unit_price, mean_price),
        :unit_cost  = coalesce(:unit_cost, mean_cost),
    )
    @aside describe(_, :nmissing) |> display
    first(10)
end

Row,variable,nmissing
,Symbol,Int64
1,product_cd,0
2,category_major_cd,0
3,category_medium_cd,0
4,category_small_cd,0
5,unit_price,0
6,unit_cost,0


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
,String15,Int64,Int64,Int64,Int64,Int64
1,P040101001,4,401,40101,198,149
2,P040101002,4,401,40101,218,164
3,P040101003,4,401,40101,230,173
4,P040101004,4,401,40101,248,186
5,P040101005,4,401,40101,268,201
6,P040101006,4,401,40101,298,224
7,P040101007,4,401,40101,338,254
8,P040101008,4,401,40101,420,315
9,P040101009,4,401,40101,498,374


In [8]:
# P-082 欠損値を中央値で補完
@chain df_products begin
    @aside median_price = round(Int, median(_.unit_price |> skipmissing))
    @aside median_cost = round(Int, median(_.unit_cost |> skipmissing))
    @rtransform(
        :unit_price = coalesce(:unit_price, median_price),
        :unit_cost = coalesce(:unit_cost, median_cost),
    )
    @aside display(describe(_, :nmissing))
    first(10)
end

Row,variable,nmissing
,Symbol,Int64
1,product_cd,0
2,category_major_cd,0
3,category_medium_cd,0
4,category_small_cd,0
5,unit_price,0
6,unit_cost,0


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
,String15,Int64,Int64,Int64,Int64,Int64
1,P040101001,4,401,40101,198,149
2,P040101002,4,401,40101,218,164
3,P040101003,4,401,40101,230,173
4,P040101004,4,401,40101,248,186
5,P040101005,4,401,40101,268,201
6,P040101006,4,401,40101,298,224
7,P040101007,4,401,40101,338,254
8,P040101008,4,401,40101,420,315
9,P040101009,4,401,40101,498,374


In [9]:
# P-083 欠損値を各商品のカテゴリ小区分コードごとの中央値で補完
stats = @chain df_products begin
    dropmissing
    groupby(:category_small_cd)
    combine(:unit_price => median, :unit_cost => median)
    @rtransform(
        :unit_price_median = round(Int, :unit_price_median),
        :unit_cost_median = round(Int, :unit_cost_median),
    )
end

@chain df_products begin
    innerjoin(stats, on= :category_small_cd)
    @rtransform(
        :unit_price = coalesce(:unit_price, :unit_price_median),
        :unit_cost = coalesce(:unit_cost, :unit_cost_median),
    )
    select(Not([:unit_price_median, :unit_cost_median]))
    @aside describe(_, :nmissing) |> display
    first(10)
end

Row,variable,nmissing
,Symbol,Int64
1,product_cd,0
2,category_major_cd,0
3,category_medium_cd,0
4,category_small_cd,0
5,unit_price,0
6,unit_cost,0


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
,String15,Int64,Int64,Int64,Int64,Int64
1,P040101001,4,401,40101,198,149
2,P040101002,4,401,40101,218,164
3,P040101003,4,401,40101,230,173
4,P040101004,4,401,40101,248,186
5,P040101005,4,401,40101,268,201
6,P040101006,4,401,40101,298,224
7,P040101007,4,401,40101,338,254
8,P040101008,4,401,40101,420,315
9,P040101009,4,401,40101,498,374


In [10]:
# P-084 各顧客の全期間の売上金額に占める2019年の割合
# 2019年とそれ以外を横持ちして集計
stats = @chain df_receipts begin
    @rtransform(:sales_year = parse(Int, string(:sales_ymd)[1:4]))
    @rtransform(:is_2019 = :sales_year == 2019)
    groupby([:customer_id, :is_2019])
    combine(:amount => sum)
    @rtransform(:is_2019 = :is_2019 ? "amount_2019" : "amount_else")
    unstack(:customer_id, :is_2019, :amount_sum)
end

@chain df_customers begin
    leftjoin(stats, on= :customer_id => :customer_id)
    @rtransform(
        :amount_2019 = coalesce(:amount_2019, 0),
        :amount_else = coalesce(:amount_else, 0),
    )
    @rtransform(:amount_all = :amount_2019 + :amount_else)
    @rtransform(:amount_2019_ratio = :amount_2019 / :amount_all)
    select(:customer_id, :amount_all, :amount_2019, :amount_2019_ratio)
    @aside describe(_, :nmissing) |> display
    first(10)
end

Row,variable,nmissing
,Symbol,Int64
1,customer_id,0
2,amount_all,0
3,amount_2019,0
4,amount_2019_ratio,0


Row,customer_id,amount_all,amount_2019,amount_2019_ratio
,String15,Int64,Int64,Float64
1,CS031415000172,5088,2971,0.583923
2,CS001215000145,875,0,0.0
3,CS015414000103,3122,874,0.279949
4,CS033513000180,868,0,0.0
5,CS011215000048,3444,248,0.0720093
6,CS040412000191,210,0,0.0
7,CS029415000023,5167,3767,0.72905
8,CS009315000023,780,0,0.0
9,CS035415000029,7504,5823,0.775986


In [11]:
# P-085 郵便番号で結合。複数紐づく場合は緯度経度を平均値の使用
latlngs = @chain df_geocodes begin
    groupby(:postal_cd)
    combine(
        :latitude => mean => :latitude,
        :longitude => mean => :longitude
    )
end

@chain df_customers begin
    innerjoin(latlngs, on= :postal_cd => :postal_cd)
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,latitude,longitude
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15,Float64,Float64
1,CS020301000012,都築 育二,0,男性,1985-08-20,33,332-0031,埼玉県川口市青木**********,S13020,20161018,0-00000000-0,35.8088,139.727
2,CS051412000011,奥村 愛,1,女性,1969-06-04,49,332-0023,埼玉県川口市飯塚**********,S13051,20180702,0-00000000-0,35.7965,139.711
3,CS051412000012,おかやま 未華子,1,女性,1977-03-13,42,332-0023,埼玉県川口市飯塚**********,S13051,20180511,0-00000000-0,35.7965,139.711
4,CS051212000001,美木 瞬,1,女性,1991-10-19,27,332-0023,埼玉県川口市飯塚**********,S13051,20180509,2-20101018-4,35.7965,139.711
5,CS020112000003,荒川 まなみ,1,女性,2004-05-13,14,332-0015,埼玉県川口市川口**********,S13020,20151116,0-00000000-0,35.8023,139.716
6,CS020212000016,藤沢 恵梨香,1,女性,1991-08-08,27,332-0015,埼玉県川口市川口**********,S13020,20150122,4-20100207-3,35.8023,139.716
7,CS051512000001,田原 夏希,1,女性,1959-11-21,59,332-0015,埼玉県川口市川口**********,S13051,20190325,0-00000000-0,35.8023,139.716
8,CS020212000004,前田 美佐,1,女性,1989-12-11,29,332-0015,埼玉県川口市川口**********,S13020,20150814,C-20090906-9,35.8023,139.716
9,CS051502000001,若山 哲平,0,男性,1961-10-07,57,332-0015,埼玉県川口市川口**********,S13051,20180209,0-00000000-0,35.8023,139.716


In [12]:
# P-086 顧客と店舗の距離を算出
latlngs = @chain df_geocodes begin
    groupby(:postal_cd)
    combine(:latitude => mean => :latitude, :longitude => mean => :longitude)
end

stores_latlngs = @chain df_stores begin
    select(:store_cd, :address => :store_address, 
        :latitude => :store_latitude, :longitude => :store_longitude)
end

@chain df_customers begin
    innerjoin(latlngs, on= :postal_cd => :postal_cd)
    innerjoin(stores_latlngs, on= :application_store_cd => :store_cd)
    @rtransform(:distance_to_store = begin
        ϕ₁, ϕ₂ = deg2rad.([:latitude, :store_latitude])
        λ₁, λ₂  = deg2rad.([:longitude, :store_longitude])
        6371 * acos(sin(ϕ₁)*sin(ϕ₂) + cos(ϕ₁)*cos(ϕ₂)*cos(λ₁ - λ₂))
    end)
    select(:customer_id, :address, :store_address, :distance_to_store)
    first(10)
end

Row,customer_id,address,store_address,distance_to_store
,String15,String,String,Float64
1,CS020301000012,埼玉県川口市青木**********,東京都北区十条仲原三丁目,4.7222
2,CS051412000011,埼玉県川口市飯塚**********,東京都板橋区大原町,3.40719
3,CS051412000012,埼玉県川口市飯塚**********,東京都板橋区大原町,3.40719
4,CS051212000001,埼玉県川口市飯塚**********,東京都板橋区大原町,3.40719
5,CS020112000003,埼玉県川口市川口**********,東京都北区十条仲原三丁目,3.94494
6,CS020212000016,埼玉県川口市川口**********,東京都北区十条仲原三丁目,3.94494
7,CS051512000001,埼玉県川口市川口**********,東京都板橋区大原町,4.1509
8,CS020212000004,埼玉県川口市川口**********,東京都北区十条仲原三丁目,3.94494
9,CS051502000001,埼玉県川口市川口**********,東京都板橋区大原町,4.1509


In [13]:
# P-087 名前と郵便番号で名寄せ。売上金額が最も高い顧客IDを残す。rank
ranked_ids = @chain df_customers begin
    leftjoin(df_receipts, on= :customer_id => :customer_id)
    groupby([:customer_id, :customer_name, :postal_cd])
    combine(:amount => sum)
    groupby([:postal_cd, :customer_name])
    @transform(:rank = ismissing(:amount_sum)
        ? competerank(:amount_sum, rev=true)
        : competerank(:customer_id))
    sort([:postal_cd, :customer_name, :rank])
end

refined_customers = @chain df_customers begin
    innerjoin(ranked_ids, on= :customer_id => :customer_id, makeunique=true)
    @rsubset(:rank == 1)
    select(names(df_customers))
end
    
string(
    "all: ",       nrow(df_customers),
    ", refined: ", nrow(refined_customers),
    ", dup: ",     nrow(@chain ranked_ids @rsubset(:rank > 1)),
) |> display
first(refined_customers, 5)

"all: 21971, refined: 21941, dup: 30"

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS037211000016,大森 杏,1,女性,1993-04-24,25,101-0025,東京都千代田区神田佐久間町**********,S13037,20150531,0-00000000-0
2,CS015301000001,米倉 聖陽,0,男性,1982-03-23,37,101-0025,東京都千代田区神田佐久間町**********,S13015,20150216,0-00000000-0
3,CS037112000007,寺西 さやか,1,女性,2004-01-20,15,101-0031,東京都千代田区東神田**********,S13037,20151108,0-00000000-0
4,CS015311000003,藤沢 知世,1,女性,1985-08-12,33,101-0031,東京都千代田区東神田**********,S13015,20141226,0-00000000-0
5,CS015301000007,大畑 秀隆,0,男性,1983-08-09,35,101-0032,東京都千代田区岩本町**********,S13015,20151014,0-00000000-0


In [14]:
# P-088
length.(unique.(
    [refined_customers.customer_id, df_customers.customer_id]))

2-element Vector{Int64}:
 21941
 21971

In [15]:
# P-089 売上実績がある顧客を8:2の割合でランダムに分割
using Random
base = @chain df_customers begin
    semijoin(df_receipts, on= :customer_id => :customer_id)
    @aside random_numbers = randperm(nrow(_))
    @transform(:random_id = random_numbers)
end

customers_20pct = @chain base @rsubset(:random_id ≤ nrow(base) * 0.2)
customers_80pct = antijoin(base, customers_20pct, on= :random_id)

# 以下、検算
# df_receipts　に　"ZZ000000000000"　というダミーの顧客IDがあり、df_customers と1件合わない。
@show nrow(base) # => 8306
@show limit = df_receipts.customer_id |> unique |> length # => 8307
@show limit * 0.2, limit * 0.8
@show antijoin(df_receipts, df_customers, on= :customer_id).customer_id |> unique

nrow.([customers_20pct, customers_80pct, df_customers]) |> display
nrow(customers_20pct) + nrow(customers_80pct)

nrow(base) = 8306
limit = (df_receipts.customer_id |> unique) |> length = 8307
(limit * 0.2, limit * 0.8) = (1661.4, 6645.6)
(antijoin(df_receipts, df_customers, on = :customer_id)).customer_id |> unique = String15["ZZ000000000000"]


3-element Vector{Int64}:
  1661
  6645
 21971

8306

In [16]:
# P-090 月ごとの金額を集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデルを構築
monthly_amounts = @chain df_receipts begin
    @rtransform(:sales_ym = string(:sales_ymd)[1:6])
    groupby(:sales_ym)
    combine(:amount => sum)
    sort(:sales_ym)
end

function split_date(df, offset_scale = 0; train_size = 12, test_size = 6, slide_window = 6)
    train_start = 1 + offset_scale * slide_window
    test_start  = train_start + train_size
    return df[train_start:test_start, :], df[test_start:(test_start + test_size), :]
end

@show df_train_1, df_test_1 = split_date(monthly_amounts)
@show df_train_2, df_test_2 = split_date(monthly_amounts, 1)
@show df_train_3, df_test_3 = split_date(monthly_amounts, 2)
;

(df_train_1, df_test_1) = split_date(monthly_amounts) = (13×2 DataFrame
 Row │ sales_ym  amount_sum
     │ String    Int64
─────┼──────────────────────
   1 │ 201701        902056
   2 │ 201702        764413
   3 │ 201703        962945
   4 │ 201704        847566
   5 │ 201705        884010
   6 │ 201706        894242
   7 │ 201707        959205
   8 │ 201708        954836
   9 │ 201709        902037
  10 │ 201710        905739
  11 │ 201711        932157
  12 │ 201712        939654
  13 │ 201801        944509, 7×2 DataFrame
 Row │ sales_ym  amount_sum
     │ String    Int64
─────┼──────────────────────
   1 │ 201801        944509
   2 │ 201802        864128
   3 │ 201803        946588
   4 │ 201804        937099
   5 │ 201805       1004438
   6 │ 201806       1012329
   7 │ 201807       1058472)
(df_train_2, df_test_2) = split_date(monthly_amounts, 1) = (13×2 DataFrame
 Row │ sales_ym  amount_sum
     │ String    Int64
─────┼──────────────────────
   1 │ 201707        959205
   2 │ 20

In [17]:
# P-091 売上実績がある顧客と無い顧客で、1:1となるようにアンダーサンプリングする
sampled_customers = @chain df_customers begin
    @aside purchased_ids = @chain _ begin
        semijoin(df_receipts, on= :customer_id => :customer_id)
        @rtransform(:purchased = true)
        select(:customer_id, :purchased)
    end
    leftjoin(purchased_ids, on= :customer_id)
    @rtransform(:purchased = ismissing(:purchased) ? false : true)
    groupby(:purchased)
    @transform(:rank = competerank(:customer_id))
    @aside limit = minimum(nrow.([purchased_ids, @rsubset(_, :purchased == false)]))
    @rsubset(:rank ≤ limit)
end

describe(sampled_customers) |> display

nrow.([@rsubset(sampled_customers, :purchased == true),
       @rsubset(sampled_customers, :purchased == false)])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,customer_id,,CS001105000001,,CS052514000001,0,String15
2,customer_name,,おかやま あさみ,,黒谷 麻緒,0,String31
3,gender_cd,1.27871,0,1.0,9,0,Int64
4,gender,,不明,,男性,0,String7
5,birth_day,,1928-11-26,1969-03-03,2007-11-25,0,Date
6,age,50.4663,11,50.0,90,0,Int64
7,postal_cd,,101-0025,,359-1132,0,String15
8,address,,千葉県佐倉市ユーカリが丘**********,,神奈川県鎌倉市西鎌倉**********,0,String
9,application_store_cd,,S12007,,S14050,0,String7


2-element Vector{Int64}:
 8306
 8306

In [18]:
# P-092 顧客の性別を第3正規形へ正規化
genders = unique(df_customers[:, [:gender, :gender_cd]])
new_customers = select(df_customers, Not(:gender))

display(genders)
first(new_customers, 3) |> display
first(rightjoin(genders, new_customers, on = :gender_cd), 3)

Row,gender,gender_cd
,String7,Int64
1,女性,1
2,不明,9
3,男性,0


Row,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,Date,Int64,String15,String,String7,Int64,String15
1,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
2,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C


Row,gender,gender_cd,customer_id,customer_name,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String7?,Int64,String15,String31,Date,Int64,String15,String,String7,Int64,String15
1,女性,1,CS021313000114,大野 あや子,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
2,不明,9,CS037613000071,六角 雅彦,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,女性,1,CS031415000172,宇多田 貴美子,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C


In [19]:
# P-093 商品データにカテゴリ名列を追加: 非正規化
new_products = @chain df_products begin
   innerjoin(df_categories, on= :category_small_cd => :category_small_cd,
        makeunique=true)
    select([names(df_products); names(df_categories)] |> unique)
end

first(new_products, 10)

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String31,String,String
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類


In [20]:
# P-094 CSV出力
mkpath("result")
CSV.write("result/P-094.csv", new_products)

"result/P-094.csv"

In [21]:
# P-095 CSV出力(cp932)
using StringEncodings
open("result/P-095.csv", enc"cp932", "w") do io
    CSV.write(io, new_products)
end

StringEncoder{UTF-8, cp932}(IOStream(<file result/P-095.csv>))

In [22]:
# P-096 CSV出力(ヘッダー行なし)
CSV.write("result/P-096.csv", new_products; header=false)

"result/P-096.csv"

In [23]:
# P-097 CSV読み込み
df = CSV.read("result/P-094.csv", DataFrame)
first(df, 3)

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String31,String,String
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類


In [24]:
# P-098 CSV読み込み(ヘッダー行なし)
df = CSV.read("result/P-096.csv", DataFrame, header=false)
first(df, 3)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,String15,Int64,Int64,Int64,Int64?,Int64?,String31,String,String
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類


In [25]:
# P-099 TSV出力
CSV.write("result/P-099.tsv", new_products; delim="\t")

"result/P-099.tsv"

In [26]:
# P-100 TSV読み込み
df = CSV.read("result/P-099.tsv", DataFrame; delim="\t")
first(df, 3)

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String31,String,String
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
